# vtab_1k

> https://google-research.github.io/task_adaptation/

https://paperswithcode.com/sota/image-classification-on-vtab-1k-1

In [1]:
#| default_exp data.vtab_1k

In [2]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [3]:
#| export
from namable_classify.data.from_ssf.vtab import VtabSplit, VtabDataset

In [38]:
VtabSplit._member_names_

['TRAIN', 'VAL', 'TEST', 'TRAIN_AND_VAL']

In [5]:
import os
vtab_dir = "/home/ycm/datasets/vtab-1k"
subset_name = "cifar"
train_dataset = VtabDataset(vtab_dir=vtab_dir, subset_name=subset_name, split=VtabSplit.TRAIN)
train_dataset

Dataset VtabDataset
    Number of datapoints: 800
    Root location: /home/ycm/datasets/vtab-1k/cifar

In [13]:
train_dataset[0]

(<PIL.Image.Image image mode=RGB size=32x32>, 66)

In [ ]:
from pathlib import Path
from typing import Callable, Optional, Union
import torchvision.datasets
from torchvision.datasets import MNIST

class VTAB_1K(torchvision.datasets.VisionDataset):
    mirrors = [
        "https://cloud.tsinghua.edu.cn/d/0d9f3b5787ab4cdab322/?dl=1",
    ]
    resources = [
        ("vtab-1k.tar.gz.part1", "b567f482832bf98fa20e29c2c11501a7"),
        ("vtab-1k.tar.gz.part2", "700f6ba9c2ddb2beeb707420f89d92b4"),
        ("vtab-1k.tar.gz.part3", "249d036285d9abff1bde741833352b22"),
        ("vtab-1k.tar.gz.part4", "57de07aec66242040573a873604125b9"),
        ("vtab-1k.tar.gz.part5", "3cc060311d5804d0db2985234e0de233"),
    ]
    def __init__(
        self,
        root: Union[str, Path],
        train: bool = True,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        download: bool = False,
    ) -> None:
        super().__init__(root, transform=transform, target_transform=target_transform)
        self.train = train  # training set or test set
        if download:
            self.download()
            
    download = MNIST.download

In [40]:
import torchvision.datasets
torchvision.datasets.MNIST?

Init signature:
torchvision.datasets.MNIST(
    root: Union[str, pathlib.Path],
    train: bool = True,
    transform: Optional[Callable] = None,
    target_transform: Optional[Callable] = None,
    download: bool = False,
) -> None
Docstring:     
`MNIST <http://yann.lecun.com/exdb/mnist/>`_ Dataset.

Args:
    root (str or ``pathlib.Path``): Root directory of dataset where ``MNIST/raw/train-images-idx3-ubyte``
        and  ``MNIST/raw/t10k-images-idx3-ubyte`` exist.
    train (bool, optional): If True, creates dataset from ``train-images-idx3-ubyte``,
        otherwise from ``t10k-images-idx3-ubyte``.
    download (bool, optional): If True, downloads the dataset from the internet and
        puts it in root directory. If dataset is already downloaded, it is not
        downloaded again.
    transform (callable, optional): A function/transform that  takes in a PIL image
        and returns a transformed version. E.g, ``transforms.RandomCrop``
    target_transform (callable, optional):

In [ ]:
from tqdm import tqdm

In [33]:
%%timeit
bar = tqdm(train_dataset)
for i, data in enumerate(bar):
    pass

100%|██████████| 800/800 [00:00<00:00, 5430.79it/s]

147 ms ± 7.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
import numpy as np
from lightning.data import optimize
from PIL import Image


# Store random images into the chunks
def images(index):
    data = {
        "index": index,
        "image": train_dataset[index][0],
        "class": train_dataset[index][1],
    }
    return data # The data is serialized into bytes and stored into chunks by the optimize operator.

if __name__ == "__main__":
    optimize(
        fn=images,  # The function applied over each input.
        inputs=list(range(len(train_dataset))),  # Provide any inputs. The fn is applied on each item.
        output_dir="vtab_1k/cifar100/train",  # The directory where the optimized data are stored.
        num_workers=4,  # The number of workers. The inputs are distributed among them.
        chunk_bytes="64MB"  # The maximum number of bytes to write into a chunk.
    )

Create an account on https://lightning.ai/ to optimize your data faster using multiple nodes and large machines.
Setting multiprocessing start_method to fork. Tip: Libraries relying on lock can hang with `fork`. To use `spawn` in notebooks, move your code to files and import it within the notebook.
Storing the files under /home/ycm/repos/novelties/cv/cls/NamableClassify/src/notebooks/02data/vtab_1k/cifar100/train
Setup started with fast_dev_run=False.
Setup finished in 0.001 seconds. Found 800 items to process.
Starting 4 workers with 800 items. The progress bar is only updated when a worker finishes.


Workers are ready ! Starting data processing...


Progress:   0%|          | 0/800 [00:00<?, ?it/s]

Rank 0 inferred the following `['int', 'pil', 'int']` data format.
Rank 1 inferred the following `['int', 'pil', 'int']` data format.Rank 2 inferred the following `['int', 'pil', 'int']` data format.Rank 3 inferred the following `['int', 'pil', 'int']` data format.


Worker 0 is terminating.
Worker 2 is terminating.Worker 1 is terminating.Worker 3 is terminating.


Worker 0 is done.
Worker 2 is done.Worker 3 is done.Worker 1 is done.


Workers are finished.
Finished data processing!


In [35]:
from lightning.data import StreamingDataset
lit_dataset = StreamingDataset(input_dir="vtab_1k/cifar100/train")

In [36]:
%%timeit
bar = tqdm(lit_dataset)
for i, data in enumerate(bar):
    pass

100%|██████████| 800/800 [00:00<00:00, 8319.97it/s]

104 ms ± 3.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()